In [9]:
!pip install transformers datasets
from transformers import BlenderbotTokenizerFast

In [3]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d christianlillelund/joe-rogan-experience-1169-elon-musk
!unzip "/content/joe-rogan-experience-1169-elon-musk.zip" -d "/content/dataset/"

Dataset URL: https://www.kaggle.com/datasets/christianlillelund/joe-rogan-experience-1169-elon-musk
License(s): CC0-1.0
  0% 0.00/59.1k [00:00<?, ?B/s]
100% 59.1k/59.1k [00:00<00:00, 46.3MB/s]
Archive:  /content/joe-rogan-experience-1169-elon-musk.zip
  inflating: /content/dataset/joe-rogan-experience-1169-elon-musk.csv  


In [30]:
!pip install -q --upgrade tensorflow

In [38]:
import pandas as pd
from datasets import load_dataset
import time
import tensorflow as tf
from transformers import DataCollatorForSeq2Seq, TFBlenderbotForConditionalGeneration, create_optimizer

ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [4]:
MAX_LENGTH=256

In [6]:
filepath="/content/dataset/joe-rogan-experience-1169-elon-musk.csv"
dataset = load_dataset('csv', data_files=filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Timestamp', 'Speaker', 'Text'],
        num_rows: 1831
    })
})

In [8]:
dataset["train"][0]

{'Timestamp': '[00:00:00]',
 'Speaker': 'Joe Rogan',
 'Text': 'Ah, ha, ha, ha. Four, three, two, one, boom. Thank you. Thanks for doing this, man. Really appreciate it.'}

In [37]:
model_id="facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizerFast.from_pretrained(model_id,truncation_side="left")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
data_array=[]
NUM_SAMPLES=8

In [14]:
tokenizer.pad_token=tokenizer.eos_token

In [16]:
for i in range(NUM_SAMPLES,len(dataset['train'])):
  discussion=""
  bot_output=tokenizer.bos_token+dataset['train'][i]["Text"]+tokenizer.eos_token
  for j in reversed(range(i-NUM_SAMPLES,i)):
    discussion=tokenizer.bos_token+dataset['train'][j]["Text"]+tokenizer.eos_token+discussion
    data_array.append([discussion,bot_output])

In [17]:
pd.DataFrame(data_array,columns=["discussion","bot_output"]).to_csv('discussion.csv')

In [18]:
df = pd.read_csv('/content/discussion.csv')

In [19]:
df

,Unnamed: 0,discussion,bot_output
0,0,"<s>Well, the flame, we didn't put a lot of tim...","<s>Now, who-</s>"
1,1,"<s>How does one, just in the middle of doing a...","<s>Now, who-</s>"
2,2,<s>You're welcome. That's coming later.</s><s>...,"<s>Now, who-</s>"
3,3,<s>And thanks for not lighting this place on f...,"<s>Now, who-</s>"
4,4,<s>Nice to meet you too.</s><s>And thanks for ...,"<s>Now, who-</s>"
...,...,...,...
14579,14579,"<s>I believe it's true too. So, thank you.</s>...","<s>Good night, everybody. END OF TRANSCRIPTAut..."
14580,14580,<s>It's true.</s><s>I believe it's true too. S...,"<s>Good night, everybody. END OF TRANSCRIPTAut..."
14581,14581,<s>And I believe it.</s><s>It's true.</s><s>I ...,"<s>Good night, everybody. END OF TRANSCRIPTAut..."
14582,14582,<s>It's true.</s><s>And I believe it.</s><s>It...,"<s>Good night, everybody. END OF TRANSCRIPTAut..."


In [20]:
filepath="/content/discussion.csv"
dataset = load_dataset('csv', data_files=filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'discussion', 'bot_output'],
        num_rows: 14584
    })
})

In [22]:
def preprocess_function(example):
  return tokenizer(
    example['discussion'],
    text_target=example['bot_output'],
    padding='max_length',
    max_length=512,
    truncation=True,)

In [23]:
tokenized_dataset=dataset.map(
    preprocess_function,remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/14584 [00:00<?, ? examples/s]

In [24]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14584
    })
})

In [ ]:
model = TFBlenderbotForConditionalGeneration.from_pretrained(model_id)

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model, return_tensors="tf")

NameError: name 'model' is not defined

In [ ]:
tf_train_dataset=tokenized_dataset["train"].to_tf_dataset(
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)

In [33]:
def replacements(a):
  for i in [1,2]:
    condition = tf.equal(a, i)
    case_true = -100*tf.ones_like(a)

    case_false = a
    a=tf.where(condition, case_true, case_false)
  return a

In [36]:
def prepare_labels(inputs):
  return {'input_ids':inputs['input_ids'],
          'attention_mask':inputs['attention_mask'],
          'labels':replacements(inputs['labels'])}

In [ ]:
train_dataset=tf_train_dataset.map(prepare_labels)

In [ ]:
model.summary()

In [ ]:
num_train_steps=len(tf_train_dataset)
optimizer, schedule = create_optimizer(
  init_lr=6e-5,
  num_warmup_steps=1_000,
  num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

In [ ]:
history=model.fit(train_dataset, epochs=1)

In [39]:
input_text=tokenizer.bos_token+"Hello Elon glad to have you on my podcast."+tokenizer.eos_token+tokenizer.bos_token+"Thanks for Having me."+tokenizer.eos_token+tokenizer.bos_token+"i heard you are building robots. Tell me more about them."+tokenizer.eos_token+tokenizer.bos_token+"Well... Currently working on a robot which can do all house chores for you "+tokenizer.eos_token+tokenizer.bos_token+"Can this robot be used in Mars?"+tokenizer.eos_token

In [ ]:
history=tokenizer(input_text, return_tensors="tf")

In [ ]:
MAX_NEW_TOKENS=16

In [ ]:
init_time=time.time()
output=model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,top_p=0.9)

print(tokenizer.decode(output[0]))
print(time.time()-init_time)

In [ ]:
init_time=time.time()
output_temp = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,temperature=1.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

In [ ]:
init_time=time.time()
output_temp = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

In [ ]:
init_time=time.time()
output_temp = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

In [ ]:
init_time=time.time()
output_topk = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

In [ ]:
init_time=time.time()
output_topk = model.generate(**history,max_new_tokens=MAX_NEW_TOKENS,do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

In [ ]:
MAX_LENGTH=1024
chat_input=""#"<s> A discussion between myself and Elon Musk who thinks his robots can get to mars</s>"

for step in range(10):
  my_text=input(">> Host:")
  new_user_input_ids = tokenizer.encode(
      tokenizer.bos_token+my_text+tokenizer.eos_token,return_tensors='tf')
  if step>0:
    chat_input=chat_input+tokenizer.bos_token+chat_history+tokenizer.eos_token+tokenizer.bos_token+my_text+tokenizer.eos_token
    bot_input_ids = tokenizer.encode(chat_input,return_tensors='tf')

  else:
    chat_input=tokenizer.bos_token+my_text+tokenizer.eos_token
    bot_input_ids = tokenizer.encode(chat_input,return_tensors='tf')

  chat_history_ids = model.generate(
      bot_input_ids,max_length=MAX_LENGTH,
      do_sample=True,
      temperature=2.0,top_k=50)

  chat_history=tokenizer.decode(chat_history_ids[0],skip_special_tokens=True,)

  print(">> Elon Musk: {}".format(tokenizer.decode(chat_history_ids[0], skip_special_tokens=True)))